In [3]:
import pandas
import numpy as np

In [4]:
df = pandas.read_csv("pokemon.csv")

names = df["name"]
names = [n.lower() for n in names] #fetches names of 801
cnames = []

for name in names:
    cnames.append(list(name))
print(names[0])
print(cnames[0])

bulbasaur
['b', 'u', 'l', 'b', 'a', 's', 'a', 'u', 'r']


In [31]:
print(cnames[0][1]+cnames[0][2])

ul


In [5]:
for name in names:
    for n in name:
        if n==" ":
            print(name)
            continue

mr. mime
mime jr.
type: null
tapu koko
tapu lele
tapu bulu
tapu fini


In [6]:
letterpairs = pandas.DataFrame(columns=["first", "second", "freq"])

for thing in cnames:
    for letter in range(len(thing)-1):
        letterpairs.loc[len(letterpairs)] = [thing[letter],thing[letter+1],1]
    letterpairs.loc[len(letterpairs)] = [thing[len(thing)-1],"END",1]

print(letterpairs)

     first second freq
0        b      u    1
1        u      l    1
2        l      b    1
3        b      a    1
4        a      s    1
...    ...    ...  ...
5975     e      a    1
5976     a      r    1
5977     r      n    1
5978     n      a    1
5979     a    END    1

[5980 rows x 3 columns]


In [35]:
dletterpairs = pandas.DataFrame(columns=["pre", "post", "freq"])

for thing in cnames:
    for letter in range(len(thing)-2):
        dletterpairs.loc[len(dletterpairs)] = [thing[letter]+thing[letter+1], thing[letter+2],1]
    dletterpairs.loc[len(dletterpairs)] = [thing[len(thing)-2]+thing[len(thing)-1],"END",1]

print(dletterpairs)


     pre post freq
0     bu    l    1
1     ul    b    1
2     lb    a    1
3     ba    s    1
4     as    a    1
...   ..  ...  ...
5174  ge    a    1
5175  ea    r    1
5176  ar    n    1
5177  rn    a    1
5178  na  END    1

[5179 rows x 3 columns]


In [7]:
print(letterpairs.loc[letterpairs["first"]==" "])

     first second freq
898             m    1
3184            j    1
5746            n    1
5846            k    1
5855            l    1
5864            b    1
5873            f    1


In [8]:
freqpairs = letterpairs.groupby(by=["first","second"], as_index=False)["freq"].sum()
print(freqpairs)

    first second  freq
0              b     1
1              f     1
2              j     1
3              k     1
4              l     1
..    ...    ...   ...
456     z      z     5
457     é    END     1
458     é      b     1
459     ♀    END     1
460     ♂    END     1

[461 rows x 3 columns]


In [36]:
dfreqpairs = dletterpairs.groupby(by=["pre","post"], as_index=False)["freq"].sum()
print(dfreqpairs)

     pre post  freq
0      b    u     1
1      f    i     1
2      j    r     1
3      k    o     1
4      l    e     1
...   ..  ...   ...
2241  zy    g     1
2242  zz  END     2
2243  zz    l     2
2244  zz    w     1
2245  éb    é     1

[2246 rows x 3 columns]


In [9]:
print(freqpairs.loc[freqpairs["second"]==" "])

    first second  freq
10      .            1
13      :            1
82      e            1
367     u            4


In [10]:
print(letterpairs)

     first second freq
0        b      u    1
1        u      l    1
2        l      b    1
3        b      a    1
4        a      s    1
...    ...    ...  ...
5975     e      a    1
5976     a      r    1
5977     r      n    1
5978     n      a    1
5979     a    END    1

[5980 rows x 3 columns]


In [11]:
freqpairs = freqpairs.drop_duplicates()
pivot_df = freqpairs.pivot(index = 'first', columns= 'second', values='freq')
sum_words = pivot_df.sum(axis=1)
pivot_df = pivot_df.apply(lambda x: x/sum_words).fillna(0)

#print(pivot_df)


In [39]:
dfreqpairs = dfreqpairs.drop_duplicates()
dpivot_df = dfreqpairs.pivot(index = 'pre', columns= 'post', values='freq')
dsum_words = dpivot_df.sum(axis=1)
dpivot_df = dpivot_df.apply(lambda x: x/dsum_words).fillna(0)

#print(dpivot_df.iloc[dpivot_df.index=="bu"])

post         '    -    .    2    :  END    a    b         c  ...         t  \
pre                                                          ...             
bu    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.043478  ...  0.130435   

post    u    v    w    x    y         z    é    ♀    ♂  
pre                                                     
bu    0.0  0.0  0.0  0.0  0.0  0.130435  0.0  0.0  0.0  

[1 rows x 36 columns]


In [12]:
print(pivot_df.iloc[pivot_df.index=="a"])

second         '    -    .    2    :       END         a         b         c  \
first                                                                          
a       0.0  0.0  0.0  0.0  0.0  0.0  0.127367  0.001721  0.034423  0.034423   

second  ...         t         u         v         w         x         y  \
first   ...                                                               
a       ...  0.060241  0.022375  0.018933  0.017212  0.008606  0.012048   

second         z    é    ♀    ♂  
first                            
a       0.012048  0.0  0.0  0.0  

[1 rows x 36 columns]


In [47]:
#Time for a fun Markov Chain
def pokeMarkovGen(s):
    newmon = []
    newmon.append(s)
    while True:
        nextchar = np.random.choice(list(pivot_df.columns), p=(pivot_df.iloc[pivot_df.index==s].values.tolist()[0]))
        newmon.append(nextchar)
        s = nextchar
        if s=="END":
            break
    #print(newmon)
    newmon = newmon[:-1]
    separator = ""
    return separator.join(newmon)

def pokeMarkovGenTwo(s):
    newmon = []
    newmon.append(s)
    while True:
        nextchar = np.random.choice(list(dpivot_df.columns), p=(dpivot_df.iloc[dpivot_df.index==s].values.tolist()[0]))
        newmon.append(nextchar)
        s = s[1]+nextchar
        if nextchar=="END":
            break
    #print(newmon)
    newmon = newmon[:-1]
    separator = ""
    return separator.join(newmon)

In [14]:
print(list(pivot_df.columns))
print(pivot_df.iloc[pivot_df.index=="a"].values.tolist())
pokeMarkovGen("b")

[' ', "'", '-', '.', '2', ':', 'END', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '♀', '♂']
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12736660929432014, 0.0017211703958691911, 0.03442340791738382, 0.03442340791738382, 0.03270223752151463, 0.0034423407917383822, 0.008605851979345954, 0.04130808950086059, 0.0, 0.024096385542168676, 0.0, 0.024096385542168676, 0.06540447504302926, 0.04475043029259897, 0.14285714285714285, 0.0017211703958691911, 0.029259896729776247, 0.0034423407917383822, 0.1703958691910499, 0.058519793459552494, 0.060240963855421686, 0.022375215146299483, 0.0189328743545611, 0.01721170395869191, 0.008605851979345954, 0.012048192771084338, 0.012048192771084338, 0.0, 0.0, 0.0]]
['b', 'w', 'n', 'c', 'n', 'n', 'm', 'n', 'l', 'n', 'n', 'm', 'w', 'l', 'l', 'r', 'b', 'END']


In [53]:
print(pokeMarkovGen("b"))
print(pokeMarkovGen("c"))
print(pokeMarkovGen("d"))
print(pokeMarkovGen("e"))
print(pokeMarkovGen("f"))

beopiloe
cicy
doatysharmanuiarar
eeakr
feligdarrare


In [66]:
print(pokeMarkovGenTwo("bu"))
print(pokeMarkovGenTwo("po"))
print(pokeMarkovGenTwo("li"))
print(pokeMarkovGenTwo("za"))
print(pokeMarkovGenTwo("go"))

butt
poltraninja
linapuff
zape
gorua


In [15]:
#Time for a less fun, but more weird alternate approach

